In [1]:
import torch
import sys
import yaml
from torchvision import transforms, datasets
import torchvision
import numpy as np
import os
from sklearn import preprocessing
from sklearn.metrics import f1_score
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [2]:
sys.path.append('../')
from models.resnet_base_network import ResNet18

In [3]:
batch_size = 256
data_transforms = torchvision.transforms.Compose([transforms.ToTensor()])

In [4]:
config = yaml.load(open("../config/config.yaml", "r"), Loader=yaml.FullLoader)

In [5]:
train_dataset = datasets.STL10('D:/kaihsiang/結構化機器學習/byol/data', split='train', download=False,
                               transform=data_transforms)

test_dataset = datasets.STL10('D:/kaihsiang/結構化機器學習/byol/data', split='test', download=False,
                               transform=data_transforms)

In [6]:
print("Input shape:", train_dataset[0][0].shape)

Input shape: torch.Size([3, 96, 96])


In [7]:
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True)

In [8]:
device = 'cpu' #'cuda' if torch.cuda.is_available() else 'cpu' 
encoder = ResNet18(**config['network'])
output_feature_dim = encoder.projetion.net[0].in_features

c:\Users\user\anaconda3\envs\fstor\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\anaconda3\envs\fstor\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [9]:
#load pre-trained parameters
load_params = torch.load(os.path.join('D:/kaihsiang/結構化機器學習/PyTorch-BYOL-master/runs/May02_20-59-00_DESKTOP-CVQR3S4/checkpoints/model.pth'),
                         map_location=torch.device(torch.device(device)))
if 'online_network_state_dict' in load_params:
    encoder.load_state_dict(load_params['online_network_state_dict'])
    print("Parameters successfully loaded.")

# remove the projection head
encoder = torch.nn.Sequential(*list(encoder.children())[:-1])    
encoder = encoder.to(device)

Parameters successfully loaded.


In [10]:
encoder

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 256

In [11]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.linear(x)

In [12]:
logreg = LogisticRegression(output_feature_dim, 10)
logreg = logreg.to(device)

In [13]:
def get_features_from_encoder(encoder, loader):
    
    x_train = []
    y_train = []

    # get the features from the pre-trained model
    for i, (x, y) in enumerate(loader):
        
        with torch.no_grad():
            feature_vector = encoder(x)
            x_train.extend(feature_vector)
            y_train.extend(y.numpy())

        
    x_train = torch.stack(x_train)
    y_train = torch.tensor(y_train)
    return x_train, y_train

In [14]:
encoder.eval()
x_train, y_train = get_features_from_encoder(encoder, train_loader)
x_test, y_test = get_features_from_encoder(encoder, test_loader)

if len(x_train.shape) > 2:
    x_train = torch.mean(x_train, dim=[2, 3])
    x_test = torch.mean(x_test, dim=[2, 3])
    
print("Training data shape:", x_train.shape, y_train.shape)
print("Testing data shape:", x_test.shape, y_test.shape)

Training data shape: torch.Size([5000, 2048]) torch.Size([5000])
Testing data shape: torch.Size([8000, 2048]) torch.Size([8000])


In [15]:
def create_data_loaders_from_arrays(X_train, y_train, X_test, y_test):

    train = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)

    test = torch.utils.data.TensorDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test, batch_size=512, shuffle=False)
    return train_loader, test_loader

In [16]:
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train).astype(np.float32)
x_test = scaler.transform(x_test).astype(np.float32)

In [17]:
train_loader, test_loader = create_data_loaders_from_arrays(torch.from_numpy(x_train), y_train, torch.from_numpy(x_test), y_test)

In [18]:
writer = SummaryWriter('./losssave/2/')

In [19]:
optimizer = torch.optim.Adam(logreg.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()
eval_every_n_epochs = 10
total_loss=0

for epoch in range(100):
#     train_acc = []
    total_loss=0
    num=0
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()        
        
        logits = logreg(x)
        predictions = torch.argmax(logits, dim=1)
        
        num+=1
        loss = criterion(logits, y)
        total_loss += loss
        loss.backward()
        optimizer.step()
    writer.add_scalar('tune_loss', total_loss.item()/num, epoch)
    
    total = 0
    if epoch % eval_every_n_epochs == 0:
        correct_top1 = 0
        correct_top5 = 0
        all_predictions = []
        all_labels = []
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)

            logits = logreg(x)
            _, predictions = logits.topk(5, dim=1, largest=True, sorted=True)

            all_predictions.extend(predictions[:, 0].cpu().numpy())
            all_labels.extend(y.cpu().numpy())

            correct_top1 += (predictions[:, 0] == y).sum().item()
            correct_top5 += (predictions == y.unsqueeze(1)).sum().item()
            total += y.size(0)
            
        f1 = f1_score(all_labels, all_predictions, average='weighted')
        top1_acc = 100 * correct_top1 / total
        top5_acc = 100 * correct_top5 / total
        print(f"Testing:Top1 accuracy: {np.mean(top1_acc)}; Top5 accuracy: {np.mean(top5_acc)}; f1-score: {np.mean(f1)}")

Testing:Top1 accuracy: 82.725; Top5 accuracy: 99.3125; f1-score: 0.8265151049220655
Testing:Top1 accuracy: 86.475; Top5 accuracy: 99.4125; f1-score: 0.8649789988180046
Testing:Top1 accuracy: 86.275; Top5 accuracy: 99.4; f1-score: 0.8630497234665592
Testing:Top1 accuracy: 86.2375; Top5 accuracy: 99.4; f1-score: 0.8628238808900648
Testing:Top1 accuracy: 85.7875; Top5 accuracy: 99.35; f1-score: 0.8582130395719008
Testing:Top1 accuracy: 85.8125; Top5 accuracy: 99.35; f1-score: 0.8584663241347193
Testing:Top1 accuracy: 85.625; Top5 accuracy: 99.3125; f1-score: 0.8566640803533571
Testing:Top1 accuracy: 85.4625; Top5 accuracy: 99.3125; f1-score: 0.854999857286134
Testing:Top1 accuracy: 85.525; Top5 accuracy: 99.2625; f1-score: 0.8555878719622271
Testing:Top1 accuracy: 85.475; Top5 accuracy: 99.2625; f1-score: 0.8550929047994215
